# 基于lora的Qwen1.8chat微调实战教程

- 日期：2024-3-16
- 作者：小知
- 基于lora参数微调Qwen1.8chat模型。

## 1.环境配置

前提: 已经配置好 `GPU` 环境。

- GPU：NVIDIA A10 cuda 11.8
- tensorflow==2.14

In [1]:
# 查看GPU
!nvidia-smi

Sat Mar 16 16:30:47 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          Off  | 00000000:00:08.0 Off |                    0 |
|  0%   27C    P8     8W / 150W |      0MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install -r requirements_qwen_1_8.txt -i https://mirrors.aliyun.com/pypi/simple

In [2]:
!pip install deepspeed transformers==4.32.0 peft pydantic==1.10.13 transformers_stream_generator einops tiktoken modelscope

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## 2.模型下载

阿里魔搭社区notebook的[jupyterLab](https://www.modelscope.cn/my/mynotebook/authorization)里：下载模型会缓存在 `/mnt/workspace/.cache/modelscope/`

In [3]:
%%time
from modelscope import snapshot_download
model_dir = snapshot_download('qwen/Qwen-1_8B-Chat')
!ls /mnt/workspace/.cache/modelscope/qwen/Qwen-1_8B-Chat/

2024-03-16 16:30:54,103 - modelscope - INFO - PyTorch version 2.1.0+cu118 Found.
2024-03-16 16:30:54,106 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2024-03-16 16:30:54,106 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-03-16 16:30:54,447 - modelscope - INFO - Loading done! Current index file version is 1.10.0, with md5 44f0b88effe82ceea94a98cf99709694 and a total number of 946 components indexed
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-16 16:30:56,478 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
Downloading: 100%|██████████| 8.21k/8.21k [00:00<00:00, 48.8MB/s]
Downloading: 100%|██████████| 50.8k/50.8k [00:00<00:00, 146MB/s]
Downloading: 100%|██████████| 244k/244k [00:00<00:00, 41.7MB/s]
D

assets				   model-00002-of-00002.safetensors
cache_autogptq_cuda_256.cpp	   modeling_qwen.py
cache_autogptq_cuda_kernel_256.cu  model.safetensors.index.json
config.json			   NOTICE.md
configuration.json		   qwen_generation_utils.py
configuration_qwen.py		   qwen.tiktoken
cpp_kernels.py			   README.md
generation_config.json		   tokenization_qwen.py
LICENSE.md			   tokenizer_config.json
model-00001-of-00002.safetensors


CPU times: user 14.6 s, sys: 8.76 s, total: 23.4 s
Wall time: 51.4 s


## 3.本地模型部署

In [14]:
%%time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig 


query = "识别以下句子中的地址信息，并按照{address:['地址']}的格式返回。如果没有地址，返回{address:[]}。句子为：在一本关于人文的杂志中，我们发现了一篇介绍北京市海淀区科学院南路76号社区服务中心一层的文章，文章深入探讨了该地点的人文历史背景以及其对于当地居民的影响。"
local_model_path = "/mnt/workspace/.cache/modelscope/qwen/Qwen-1_8B-Chat/"
tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(local_model_path, device_map="auto", trust_remote_code=True).eval()
response, history = model.chat(tokenizer, query, history=None)
print("回答如下:\n", response)


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.11it/s]


回答如下:
 在这个句子中，有三个地址信息：
1. 北京市海淀区科学院南路76号社区服务中心一层。
2. 文章深入探讨了该地点的人文历史背景以及其对于当地居民的影响。

按照{address:['地址']}的格式返回：
在一本关于人文的杂志中，我们发现了一篇介绍北京市海淀区科学院南路76号社区服务中心一层的文章，文章深入探讨了该地点的人文历史背景以及其对于当地居民的影响。
CPU times: user 3.51 s, sys: 280 ms, total: 3.79 s
Wall time: 3.79 s


## 4.下载Qwen仓库

克隆Qwen项目，调用`finetune.py`文件进行微调。

In [5]:
%%time
!git clone https://gitcode.com/QwenLM/Qwen.git

正克隆到 'Qwen'...
remote: Enumerating objects: 1458, done.
remote: Total 1458 (delta 0), reused 0 (delta 0), pack-reused 1458
接收对象中: 100% (1458/1458), 35.31 MiB | 44.42 MiB/s, 完成.
处理 delta 中: 100% (855/855), 完成.
CPU times: user 19.4 ms, sys: 22.8 ms, total: 42.3 ms
Wall time: 1.82 s


## 5.微调与配置

调用`Qwen/finetune.py`文件进行配置与微调。


- --model_name_or_path Qwen-1_8B-Chat：指定预训练模型的名称或路径，这里是使用名为"Qwen-1_8B-Chat"的预训练模型。
- --data_path chat.json：指定训练数据和验证数据的路径，这里是使用名为"chat.json"的文件。
- --fp16 True：指定是否使用半精度浮点数（float16）进行训练，这里设置为True。
- --output_dir output_qwen：指定输出目录，这里是将训练结果保存到名为"output_qwen"的文件夹中。
- --num_train_epochs 5：指定训练的轮数，这里是训练5轮。
- --per_device_train_batch_size 2：指定每个设备（如GPU）上用于训练的批次大小，这里是每个设备上训练2个样本。
- --per_device_eval_batch_size 1：指定每个设备上用于评估的批次大小，这里是每个设备上评估1个样本。
- --gradient_accumulation_steps 8：指定梯度累积步数，这里是梯度累积8步后再更新模型参数。
- --evaluation_strategy "no"：指定评估策略，这里是不进行评估。
- --save_strategy "steps"：指定保存策略，这里是每隔一定步数（如1000步）保存一次模型。
- --save_steps 1000：指定保存步数，这里是每隔1000步保存一次模型。
- --save_total_limit 10：指定最多保存的模型数量，这里是最多保存10个模型。
- --learning_rate 3e-4：指定学习率，这里是3e-4。
- --weight_decay 0.1：指定权重衰减系数，这里是0.1。
- --adam_beta2 0.95：指定Adam优化器的beta2参数，这里是0.95。
- --warmup_ratio 0.01：指定预热比例，这里是预热比例为总步数的1%。
- --lr_scheduler_type "cosine"：指定学习率调度器类型，这里是余弦退火调度器。
- --logging_steps 1：指定日志记录步数，这里是每1步记录一次日志。
- --report_to "none"：指定报告目标，这里是不报告任何信息。
- --model_max_length 512：指定模型的最大输入长度，这里是512个字符。
- --lazy_preprocess True：指定是否使用懒加载预处理，这里设置为True。
- --gradient_checkpointing：启用梯度检查点技术，可以在训练过程中节省显存并加速训练。
- --use_lora：指定是否使用LORA（Layer-wise Relevance Analysis）技术，这里设置为True

In [6]:
%%time
!python ./Qwen/finetune.py \
--model_name_or_path "/mnt/workspace/.cache/modelscope/qwen/Qwen-1_8B-Chat/" \
--data_path qwen_chat.json \
--fp16 True \
--output_dir output_qwen \
--num_train_epochs 10 \
--per_device_train_batch_size 2 \
--per_device_eval_batch_size 1 \
--gradient_accumulation_steps 8 \
--evaluation_strategy "no" \
--save_strategy "steps" \
--save_steps 1000 \
--save_total_limit 10 \
--learning_rate 3e-4 \
--weight_decay 0.1 \
--adam_beta2 0.95 \
--warmup_ratio 0.01 \
--lr_scheduler_type "cosine" \
--logging_steps 1 \
--report_to "none" \
--model_max_length 512 \
--lazy_preprocess True \
--gradient_checkpointing True \
--use_lora True

[2024-03-16 16:32:02,034] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2024-03-16 16:32:03.298260: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-16 16:32:03.328849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 16:32:03.328873: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 16:32:03.328894: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for

## 6.模型合并

In [7]:
%%time
from peft import AutoPeftModelForCausalLM 
from transformers import AutoTokenizer 


# 分词
tokenizer = AutoTokenizer.from_pretrained("output_qwen", trust_remote_code=True ) 
tokenizer.save_pretrained("qwen-1_8b-finetune")

# 模型
model = AutoPeftModelForCausalLM.from_pretrained("output_qwen", device_map="auto", trust_remote_code=True ).eval() 
merged_model = model.merge_and_unload() 
merged_model.save_pretrained("qwen-1_8b-finetune", max_shard_size="2048MB", safe_serialization=True) # 最大分片2g



The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.43it/s]


CPU times: user 10.2 s, sys: 3.06 s, total: 13.2 s
Wall time: 12.7 s


## 7.本地部署微调模型

In [11]:
%%time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig 


query = "识别以下句子中的地址信息，并按照{address:['地址']}的格式返回。如果没有地址，返回{address:[]}。句子为：在一本关于人文的杂志中，我们发现了一篇介绍北京市海淀区科学院南路76号社区服务中心一层的文章，文章深入探讨了该地点的人文历史背景以及其对于当地居民的影响。"
local_model_path = "qwen-1_8b-finetune"
tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(local_model_path, device_map="auto", trust_remote_code=True).eval()
response, history = model.chat(tokenizer, query, history=None)
print("回答如下:\n", response)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.03it/s]


回答如下:
 {"address":"北京市海淀区科学院南路76号社区服务中心一层"}
CPU times: user 1.66 s, sys: 269 ms, total: 1.93 s
Wall time: 1.93 s


## 8.保存依赖包信息

In [13]:
!pip freeze > requirements_qwen_1_8.txt

## 参考资料

- https://www.modelscope.cn/models/qwen/Qwen-1_8B-Chat/summary
- https://gitcode.com/QwenLM/Qwen.git
- https://blog.csdn.net/qq_45156060/article/details/135153920
- https://blog.csdn.net/weixin_44750512/article/details/135099562